# 한-영 번역기
한-영 번역기에 오신 것을 환영합니다!

In [2]:
%pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
from transformers import (
    EncoderDecoderModel,
    PreTrainedTokenizer,
    PreTrainedTokenizerFast,
    BertTokenizer,
)

In [6]:
%pwd

'/root/test-1/baseline'

In [8]:
src_tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
trg_tokenizer = PreTrainedTokenizerFast.from_pretrained('skt/kogpt2-base-v2',
  bos_token='</s>', eos_token='</s>', unk_token='<unk>',
  pad_token='<pad>', mask_token='<mask>')

model = EncoderDecoderModel.from_pretrained('/root/test-1/dump/best_model')
model.eval()
model.cuda()
model.config.decoder_start_token_id = trg_tokenizer.bos_token_id

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [5]:
from IPython.display import display
import ipywidgets as widgets

kor = widgets.Textarea(
    placeholder='번역할 한국어',
    description="입력",
    disabled=False
)

button = widgets.Button(
    description='번역!',
    disabled=False,
    tooltip='해당 한국어 입력을 번역합니다.'
)

eng = widgets.Textarea(
    description="출력",
    disabled=True
)

def translate(_):
    eng.value = ""
    text = kor.get_interact_value()
    embeddings = src_tokenizer(text, return_attention_mask=False, return_token_type_ids=False, return_tensors='pt')
    embeddings = {k: v.cuda() for k, v in embeddings.items()}
    output = model.generate(**embeddings)[0, 1:-1]
    eng.value = trg_tokenizer.decode(output.cpu())

button.on_click(translate)
display(kor, button, eng)

Textarea(value='', description='입력', placeholder='번역할 한국어')

Button(description='번역!', style=ButtonStyle(), tooltip='해당 한국어 입력을 번역합니다.')

Textarea(value='', description='출력', disabled=True)